# DICOM-extraction
This notebook outlines the procedure for extracting the desired MR-sequences from Philips' enhanced DICOM format to Niftii. The DICOM-files are not included, but we still provide this notebook for transparency. The MRI-data were located within it's own folder in the `mri_dataset`-directory, labeled `sourcedata`, according to a structure `sourcedata/sub-XX/ses-XX/YYYY_MM_DD/{SESSION ID}/DICOM_A_B_{SEQUENCE_LABEL}/DICOM/IM_000X`. 
At various levels in the aformentioned path-three, there might also be additional files or directories that we want to ignore.

There are 7 different types of MRI-sequences included in this dataset:
- T1w: All sessions
- LookLocker: All sessions
- Mixed: All sessions
- FLAIR: Pre-contrast only
- T2w: Pre-contrast only
- DTI dynamic: Pre-contrast only
- DTI Top-Up Prescan: Pre-contrast only

All of the sequences except the Mixed IR-SE sequence is converted from DICOM to Nifti using `dcm2niix`. For further details on the volume conversion from the mixed sequence conversion, see ()[] or the script `extract_mixed_sequence.py`.
The Look-Locker sequence if further manipulated to include all 14 folumes from different triggering times into a single 4D-volume. See below for further details. 

In [ ]:
import subprocess
import tempfile
import itertools
import warnings
import re
import json
import shutil

from datetime import date, time, datetime
from pathlib import Path

import pandas as pd
import pydicom
import nibabel
import numpy as np

import sys
sys.path.append("../src/gonzo")

from gmri2fem.mixed_dicom import dcm2nii_mixed
from loguru import logger
logger.disable("mixed_dicom")


NUM_SESSIONS = 5
mri_dataset = Path("../mri_dataset")
private_meta = mri_dataset / "participants-private.json"
sourcedata = mri_dataset / "sourcedata"


sub_re = r"(?P<subject>sub-(control|patient)*\d{2})"
ses_re = r"(?P<session>ses-(\d{2}))"
date_re = r"(?P<date>\d{4}_\d{2}_\d{2})"
session_id_re = r"(?P<session_id>\w+)"
seq_folder_re = r"(?P<sequence_folder>(DICOM_(?P<scan_id>\d+)_(?P<series_id>\d)[_ ](?P<sequence_label>.+)))"

subject_re = re.compile(sub_re)
subject_paths = sorted(filter(lambda x: subject_re.match(x.name), sourcedata.glob("*")))
subjects = [p.name for p in subject_paths]
subjects

In [ ]:
def contrast_injection_datetime(subject: str, private_meta: Path) -> datetime:
    with open(private_meta, "r") as f:
        injection_time = [
            datetime.strptime(entry["injection_time"], "%Y%m%d_%H%M%S")
            for entry in json.load(f) if entry["participant_id"] == subject 
        ][0]
    return injection_time

def sequence_acquisition_date(sequence_path: Path | str) -> datetime:
    date_regex = re.compile("(?P<year>\d{4})_(?P<month>\d{2})_(?P<day>\d{2})")
    m = date_regex.search(str(sequence_path))
    if m is None:
        raise ValueError(f"No date found in '{sequence_path}'")
    return datetime(**{key: int(val) for key, val in m.groupdict().items()}).date()


def sequence_acquisition_time_of_day(sequence_outpath: Path) -> time:
    sidecars = (sequence_outpath.parent).rglob(f"*{sequence_outpath.stem}*.json")
    for sidecar in sidecars:        
        with open(sidecar, "r") as f:
            info = json.load(f)
        if "AcquisitionTime" in info:
            return datetime.strptime(info["AcquisitionTime"], "%H:%M:%S.%f").time()
    raise RuntimeError(f"Couldn't find sidecar with 'AcquisitionTime' in {sequence_outpath}")

def sequence_acquisition_datetime(sequence_path: Path, sequence_outpath: Path) -> datetime:
    return datetime.combine(
        sequence_acquisition_date(sequence_path),
        sequence_acquisition_time_of_day(sequence_outpath)
    )


def sequence_seconds_relative_injection(subject, private_meta, sequence_path, sequence_outpath) -> int:
    return (
        sequence_acquisition_datetime(sequence_path, sequence_outpath)
        - contrast_injection_datetime(subject, private_meta)
    ).total_seconds()

In [ ]:
class Sequence:
    def __init__(self, name, search_pattern, dtype, label, ignore_pattern=None, sessions=None, overwrite=False, label_shortform=None):
        self.name = name
        self.search_pattern = search_pattern
        if ignore_pattern is None:
            self.ignore_pattern = re.compile("(NOCONVERT|IGNORE)")
        else:
            self.ignore_pattern = ignore_pattern
        self.dtype = dtype
        self.label = label
        self.overwrite = overwrite
        if sessions is not None:
            self.expected_sessions = sessions
        else:
            self.expected_sessions = [f"ses-{idx:02d}" for idx in range(1, NUM_SESSIONS+1)]
        if label_shortform is None:
            self.label_shortform = label
        else:
            self.label_shortform = label_shortform
            
    def outpath(self, dataset, subject, session):
        return dataset / subject / session / self.dtype / f"{subject}_{session}_{self.label}"

    def convert(self, dicomfile, outpath, additional_args=None):
        if additional_args is None:
            additional_args = ""
        outdir, form = outpath.parent, outpath.stem
        outdir.mkdir(exist_ok=True, parents=True)
        cmd = f"dcm2niix -f {form} -z y -o '{outdir}' {additional_args} '{dicomfile}' >> /tmp/dcm2niix.txt"
        try:
            subprocess.run(cmd, shell=True).check_returncode()
        except (ValueError, subprocess.CalledProcessError) as e:
            print(str(e))
            pass
        
    def multiple_found_sequences(self, seq_folders):
        print(f"Found multiple directories in {session_folder} matching {seq.search_pattern.pattern}, ignoring.")
        return []
        
class LookLockerSequence(Sequence):
    def convert(self, dicomfile, outpath):
        outdir, form = outpath.parent, outpath.stem
        outdir.mkdir(exist_ok=True, parents=True)
        with tempfile.TemporaryDirectory(prefix=outpath.stem) as tmpdir:
            tmppath = Path(tmpdir)
            super().convert(dicomfile, tmppath / "ll_split")
            timestamp_re = re.compile("ll_split_t(?P<timestamp>\d+).json")
            groups = [timestamp_re.match(p.name) for p in tmppath.glob("*.json")]
            time_ms = sorted([int(g["timestamp"]) for g in groups])
            nii_files = [tmppath / f"ll_split_t{t}.nii.gz" for t in time_ms]
            example_nii = nibabel.nifti1.load(nii_files[0])
            affine = example_nii.affine
            D = np.empty((*example_nii.shape, len(nii_files)), dtype=np.single)
            for idx, nii in enumerate(nii_files):
                D[..., idx] = nibabel.nifti1.load(nii).get_fdata(dtype=np.single)
            shutil.copy(tmppath / f"ll_split_t{time_ms[0]}.json", outpath.with_suffix(".json"))

        
        np.savetxt(outpath.with_name(outpath.stem + "_trigger_times.txt"), time_ms)
        nibabel.nifti1.save(
            nibabel.nifti1.Nifti1Image(D, affine),
            outpath.with_suffix(".nii.gz")
        )
        
class MixedSequence(Sequence):
    def convert(self, dicomfile, outpath):
        outdir, form = outpath.parent, outpath.stem
        outdir.mkdir(exist_ok=True, parents=True)
        IR_out = outpath.with_name(outpath.stem + "_IR-corrected-real")        
        SE_out = outpath.with_name(outpath.stem + "_SE-modulus")
        T1_out = outpath.with_name(outpath.stem + "_T1map_scanner")

        
        IR_data, SE_data, T1_data = dcm2nii_mixed(next(dicomfile.rglob("IM_*")), ["IR-corrected-real", "SE-modulus", "T1map-scanner"])
        nibabel.nifti1.save(IR_data["nifti"], IR_out.with_suffix(".nii.gz"))
        nibabel.nifti1.save(SE_data["nifti"], SE_out.with_suffix(".nii.gz"))
        nibabel.nifti1.save(T1_data["nifti"], T1_out.with_suffix(".nii.gz"))
        print(T1_data)
                                                                         
        meta = {
            "TR_SE": SE_data["descrip"]["TR"],
            "TE": SE_data["descrip"]["TE"],
            "TR_IR": IR_data["descrip"]["TR"],
            "TI": IR_data["descrip"]["TI"],
        }
        with open(IR_out.parent / f"{form}_meta.json", "w") as f:
            json.dump(meta, f)

        try:
            cmd = f"dcm2niix -w 0 --terse -b o -f '{form}' -o '{outdir}' '{dicomfile}' >> /tmp/dcm2niix.txt "
            subprocess.run(cmd, shell=True).check_returncode()
        except (ValueError, subprocess.CalledProcessError) as e:
            print(str(e))
            pass
        
    def multiple_sequences(self, seq_folders):
        new_folders = list(filter(lambda p: re.search("DelRec", str(p)), seq_folders))
        if len(new_folders) == 0:
            print(f"Found multiple folders matching {self.regex.pattern}, but none containing 'DelRec': {seq_folders}")
        elif len(new_folders) > 1:
            print(f"Found multiple folders matching {self.regex.pattern} and contatining 'DelRec': {seq_folders}. Ignoring.")
            return []
        return new_folders

        

In [ ]:
overwrite = True
sequences = [
    Sequence(
        name="T1w",
        search_pattern=re.compile("T1_3D"),
        dtype="anat",
        label="T1w",
        overwrite=overwrite
    ),
    Sequence(
        name="T2w",
        search_pattern=re.compile("DICOM_\d+_1.*(TE565|T2W).*"),
        dtype="anat",
        label="T2w",
        overwrite=overwrite,
        sessions=["ses-01"]
    ),
    Sequence(
        name="FLAIR",
        search_pattern=re.compile(".*FLAIR[_ ]3D.*"),
        dtype="anat",
        label="FLAIR",
        overwrite=overwrite,
        sessions=["ses-01"]
    ),
    Sequence(
        name="dDTI",
        search_pattern = re.compile(".*DTI.*dynamisk.*"),
        dtype="dwi",
        label="acq-multiband_sense_dir-AP_DTI",
        label_shortform="dDTI",
        overwrite=overwrite,
        sessions=["ses-01"]
    ),
    Sequence(
        name="dDTI-Topup",
        search_pattern = re.compile(".*DTI.*(Top-Up|top-up).*"),
        dtype="dwi",
        label="acq-multiband_sense_dir-PA_b0",
        label_shortform="dDTI-prescan",
        overwrite=overwrite,
        sessions=["ses-01"]
    ),
    MixedSequence(
        name="Mixed",
        search_pattern = re.compile("DICOM_\d+_\d.*Mixed.*"),
        dtype="mixed",
        label="acq-mixed",
        label_shortform="mixed",
        overwrite=True
    ),
    LookLockerSequence(
        name="LookLocker",
        search_pattern = re.compile("DICOM_\d+_3.*(LookLocker|2beatpause)"),
        dtype="anat",
        label="acq-looklocker_IRT1",
        label_shortform="looklocker",
        overwrite=overwrite
    ),
]
date_regex = re.compile(date_re)
seq_folder_regex = re.compile(seq_folder_re)
sessions = [f"ses-{j+1:02d}" for j in range(NUM_SESSIONS)]
timetable_records = []
for subject in subjects:
    print("=" * 40, subject, "=" * 40)
    
    # Ensure the subjects sourcedata exsists
    subject_path = (sourcedata / subject)
    if not (sourcedata / subject).exists():
        print(f"Missing subject folder {sourcedata/subject}.")
        continue
    for session in sessions:
        # Ensure sourcedata of given session exists
        session_path = subject_path / session
        if not session_path.exists():
            print(f"Missing session folder {session_path}.")
            continue

        # Ensure session has the expected directory structure, and find the date
        datefolders = sorted(filter(lambda x: date_regex.match(x.name), session_path.iterdir()))
        assert len(datefolders) == 1, f"Session should contain a single date-folder, found {list(session_path.iterdir())}. Choosing first: {datefolders[0]}"
        datefolder = datefolders[0]
        session_date = datetime.strptime(datefolder.name, "%Y_%m_%d").date()

        
        # Uncover cases where several scans were performed during the same session
        session_folders = sorted(datefolder.glob("*"))
        if len(session_folders)  == 0:
            print(f"Found no session-id folder in {datefolder}")
            continue
        elif len(session_folders) > 1:
            print(f"Found several session-id folders in {datefolder}: {session_folders}. Choosing first: {session_folders[0]}")
        session_folder = session_folders[0]
        all_seq_folders = [p for p in sorted(filter(lambda x: seq_folder_regex.match(x.name), session_folder.glob("*")))]

        for seq in sequences:
            # Check if the output folder already contains files with desired output pattern, and optionally delete.
            seq_out = seq.outpath(mri_dataset, subject, session)
            seq_match_in_target = sorted(
                filter(
                    lambda x: re.match(f"{seq_out.stem}(.*)\.(nii\.gz|json)", x.name) is not None,
                    seq_out.parent.glob("*")
                )
            )
            if len(seq_match_in_target) > 0:
                if not seq.overwrite:
                    continue
                else:
                    for file in seq_match_in_target:
                        file.unlink()

            
            # Uncover either missing or multiple sequences matching pattern to investigate potential
            # data error, or if the regex needs to be adjusted.
            seq_folders = [
                folder for folder in all_seq_folders if (
                    seq.search_pattern.search(folder.name) and (not seq.ignore_pattern.search(folder.name))
                )
            ]
            if len(seq_folders) == 0:
                if session in seq.expected_sessions:
                    print(f"Couldn't find pattern {seq.search_pattern.pattern} in {session_folder}")
                continue
            elif len(seq_folders) > 1:
                print(f"Found mulitple directories in {session_folder} matching {seq.search_pattern.pattern}, ignoring.")
                continue
            seq_folder = seq_folders[0]
            if session not in seq.expected_sessions:
                print(f"Found sequence {seq.name} with pattern {seq.search_pattern} in unexpected session {session}: Extracting to ses-01")
                # Check if the output folder already contains files with desired output pattern        
                seq_out = seq.outpath(mri_dataset, subject, "ses-01")
            seq.convert(seq_folder , seq_out)

            relative_time = sequence_seconds_relative_injection(subject, private_meta, seq_folder, seq_out)
            timetable_record = {
                "subject": subject,
                "session": session,
                "sequence_label": seq.label_shortform,
                "acquisition_relative_injection": relative_time
            }
            timetable_records.append(timetable_record)

        print()

In [ ]:
dframe = pd.DataFrame.from_records(timetable_records).sort_values(by=["subject", "sequence_label", "session", "acquisition_relative_injection"])
if (mri_dataset / "timetable.tsv").exists():
    answer = input("Timetable already exists. Overwrite?")
    if answer.lower() in ["y", "yes"]:
        dframe.to_csv(mri_dataset / "timetable.tsv", index=False, sep="\t")
else:
    dframe.to_csv(mri_dataset / "timetable.tsv", index=False, sep="\t")

pd.read_csv(mri_dataset / "timetable.tsv", sep="\t")